
# Laboratorio 11 - Visualización Interactiva con Plotly

### Javier Ovalle, José Ángel Morales, Ricardo Morales
Link del repositorio: https://github.com/rick-jmc/Laboratorio-11---Data-Science

Este cuaderno implementa un **tablero de control interactivo** (en Jupyter) usando **Plotly** y widgets de Jupyter, cumpliendo los requisitos del laboratorio:  
- Mínimo **8 visualizaciones interactivas**, al menos **2 enlazadas**.  
- Permite al usuario **explorar**, **seleccionar vistas** y **cambiar el nivel de detalle**.  
- Incluye **3+ modelos simples de predicción** para series temporales y comparación de desempeño.  
- Diseño consistente con la **paleta de colores** definida por el equipo.

> Nota: Este cuaderno está pensado para ejecutarse localmente. Asegúrese de instalar dependencias en la primera celda.



## Paleta de colores y estilo

Se utiliza la paleta definida por el equipo (cálidos + neutros):
- Rojo petróleo: **#C0392B**
- Naranja ámbar: **#E67E22**
- Amarillo dorado: **#F1C40F**
- Gris grafito: **#2C3E50**
- Blanco humo: **#ECF0F1**

Estos colores se aplican mediante un **template** de Plotly para reforzar consistencia visual (fondos, tipografías, ejes y secuencias categóricas).


In [ ]:

# --- Instalación de dependencias (ejecutar una sola vez si es necesario) ---
# Si su entorno ya cuenta con estos paquetes, puede omitir esta celda.
import sys, subprocess

def pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

need = [
    "plotly>=5.22.0",
    "pandas>=2.0.0",
    "numpy",
    "scikit-learn",
    "statsmodels",
    "ipywidgets"
]
for pkg in need:
    try:
        __import__(pkg.split("==")[0].split(">=")[0])
    except Exception:
        pip_install(pkg)

# Activar el renderer dentro del notebook
import plotly.io as pio
# Ajuste automático recomendado en Jupyter Lab/Notebook:
pio.renderers.default = "notebook"
print("Dependencias listas y renderer configurado:", pio.renderers.default)


In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, learning_curve
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Template / tema corporativo (paleta del equipo)
UVG_COLORS = ["#C0392B", "#E67E22", "#F1C40F", "#2C3E50", "#ECF0F1"]
BG = "#ECF0F1"   # fondo
FG = "#2C3E50"   # texto/ejes

uvg_template = dict(
    layout=dict(
        paper_bgcolor=BG,
        plot_bgcolor="white",
        font=dict(family="Arial", size=12, color=FG),
        title=dict(font=dict(color=FG, size=18)),
        legend=dict(bgcolor="rgba(0,0,0,0)", borderwidth=0),
        xaxis=dict(gridcolor="#D0D3D4", zerolinecolor="#D0D3D4"),
        yaxis=dict(gridcolor="#D0D3D4", zerolinecolor="#D0D3D4"),
        colorway=UVG_COLORS[:4],  # prioritariamente cálidos y gris
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
        margin=dict(l=60, r=30, t=60, b=50)
    )
)
px.defaults.template = uvg_template
go.layout.Template(uvg_template)  # for completeness
print("Template UVG aplicado.")


In [ ]:

# --- Carga de datos ---
# Ajuste el path si el archivo se encuentra en otra ruta
EXCEL_PATH = "/mnt/data/IMPORTACIONES.xlsx"

# 1) Lectura y limpieza básica
xls = pd.ExcelFile(EXCEL_PATH)
df = xls.parse(xls.sheet_names[0]).copy()

# Normalizar nombres de columnas (strip y espacios)
df.columns = [c.strip() for c in df.columns]

# Convertir Fecha a datetime y crear atributos temporales
df["Fecha"] = pd.to_datetime(df["Fecha"])
df["Año"] = df["Fecha"].dt.year
df["Mes"] = df["Fecha"].dt.month
df["MesNombre"] = df["Fecha"].dt.month_name(locale="es_ES") if hasattr(pd.Series.dt, "month_name") else df["Fecha"].dt.month

# 2) Construcción de categorías relevantes
diesel_cols = [c for c in df.columns if c.lower().startswith("diesel")]
fuel_map = {
    "Diesel (total)": df[diesel_cols].sum(axis=1) if diesel_cols else 0.0,
    "Gasolina regular": df.get("Gasolina regular", pd.Series([0.0]*len(df))),
    "Gasolina superior": df.get("Gasolina superior", pd.Series([0.0]*len(df))),
    "Gas licuado de petróleo": df.get("Gas licuado de petróleo", pd.Series([0.0]*len(df)))
}

fuels_df = pd.DataFrame(fuel_map)
fuels_df["Fecha"] = df["Fecha"]
fuels_df["Año"] = df["Año"]
fuels_df["Mes"] = df["Mes"]

# DataFrame "largo" para gráficas flexibles
tidy = fuels_df.melt(id_vars=["Fecha", "Año", "Mes"], var_name="Combustible", value_name="Importaciones")
tidy = tidy.sort_values("Fecha").reset_index(drop=True)

# Totales por fecha (todas las categorías del archivo, si hay una columna 'Total importación' úsela)
if "Total importación" in df.columns:
    total_series = df[["Fecha", "Total importación"]].rename(columns={"Total importación": "Total"})
else:
    # Fallback: suma de todas las columnas no temporales (esto incluye múltiples productos)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if "Año" in numeric_cols: numeric_cols.remove("Año")
    if "Mes" in numeric_cols: numeric_cols.remove("Mes")
    total_series = pd.DataFrame({"Fecha": df["Fecha"], "Total": df[numeric_cols].sum(axis=1)})

print("Filas:", len(df), "| Columnas:", len(df.columns))
display(df.head(3))
display(tidy.head(6))


In [ ]:

# -----------------------------
# Utilidades: Modelado y Métricas
# -----------------------------
from typing import Dict, Tuple

def split_series(ts: pd.Series, test_months: int = 12) -> Tuple[pd.Series, pd.Series]:
    ts = ts.dropna()
    if len(ts) <= test_months:
        raise ValueError("Serie demasiado corta para partición train/test.")
    train = ts.iloc[:-test_months]
    test  = ts.iloc[-test_months:]
    return train, test

def metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = (np.abs((y_true - y_pred) / np.where(y_true==0, np.nan, y_true))).mean() * 100
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "MAPE%": mape, "R2": r2}

def forecast_naive_seasonal(train: pd.Series, test: pd.Series, seasonal_lag: int = 12) -> np.ndarray:
    # Pronóstico = valor de hace 12 meses (si no hay, usa último valor disponible)
    preds = []
    history = train.copy()
    for i in range(len(test)):
        if len(history) >= seasonal_lag:
            preds.append(history.iloc[-seasonal_lag])
        else:
            preds.append(history.iloc[-1])
        history = pd.concat([history, pd.Series([test.iloc[i]], index=[test.index[i]])])
    return np.array(preds)

def forecast_moving_average(train: pd.Series, test: pd.Series, window: int = 3) -> np.ndarray:
    last_ma = train.rolling(window=window, min_periods=1).mean().iloc[-1]
    preds = np.full(shape=len(test), fill_value=last_ma)
    return preds

def forecast_holt_winters(train: pd.Series, test: pd.Series) -> np.ndarray:
    # Modelo aditivo simple con estacionalidad mensual (12)
    model = ExponentialSmoothing(train, trend="add", seasonal="add", seasonal_periods=12, initialization_method="estimated")
    fit = model.fit(optimized=True, use_brute=True)
    fh = fit.forecast(steps=len(test))
    return np.array(fh)

def forecast_random_forest(train: pd.Series, test: pd.Series, start_date: pd.Timestamp) -> np.ndarray:
    # Construimos features de calendario (año, mes, sin/cos estacionales)
    # Índices mensuales consecutivos a partir de start_date
    X_all = pd.DataFrame({"Fecha": pd.date_range(start=start_date, periods=len(train)+len(test), freq="MS")})
    X_all["Año"] = X_all["Fecha"].dt.year
    X_all["Mes"] = X_all["Fecha"].dt.month
    X_all["mes_sin"] = np.sin(2*np.pi*X_all["Mes"]/12)
    X_all["mes_cos"] = np.cos(2*np.pi*X_all["Mes"]/12)

    y_all = pd.concat([train, test]).reset_index(drop=True)

    X_train = X_all.iloc[:len(train), 1:]
    X_test  = X_all.iloc[len(train):, 1:]
    rf = RandomForestRegressor(n_estimators=300, random_state=42)
    rf.fit(X_train, train.values)
    preds = rf.predict(X_test)
    return np.array(preds)

def make_learning_curve(series: pd.Series, start_date: pd.Timestamp):
    # Learning curve para RandomForest (regresión). Devuelve fig Plotly.
    n = len(series)
    X = pd.DataFrame({"Fecha": pd.date_range(start=start_date, periods=n, freq="MS")})
    X["Año"] = X["Fecha"].dt.year
    X["Mes"] = X["Fecha"].dt.month
    X["mes_sin"] = np.sin(2*np.pi*X["Mes"]/12)
    X["mes_cos"] = np.cos(2*np.pi*X["Mes"]/12)
    y = series.values

    tscv = TimeSeriesSplit(n_splits=5)
    train_sizes, train_scores, test_scores = learning_curve(
        RandomForestRegressor(n_estimators=250, random_state=42),
        X[["Año","Mes","mes_sin","mes_cos"]], y,
        cv=tscv, scoring="r2", train_sizes=np.linspace(0.3, 1.0, 6), n_jobs=None
    )
    fig = go.Figure()
    fig.add_scatter(x=train_sizes, y=train_scores.mean(axis=1), mode="lines+markers", name="Entrenamiento")
    fig.add_scatter(x=train_sizes, y=test_scores.mean(axis=1), mode="lines+markers", name="Validación")
    fig.update_layout(title="Learning curve (RandomForest, R²)",
                      xaxis_title="Tamaño de entrenamiento",
                      yaxis_title="R²")
    return fig



## Exploración interactiva

A continuación se presentan varias visualizaciones **interactivas**.  
- Puede **seleccionar combustibles** y **rangos de fechas**.  
- Hay **gráficos enlazados**: seleccione años/fechas para actualizar vistas dependientes.  
- Use los **controles** (dropdowns, checkboxes) para mostrar u ocultar visualizaciones.


In [ ]:

# Widget: selección de combustibles
fuel_options = tidy["Combustible"].unique().tolist()
fuel_multi = widgets.SelectMultiple(
    options=fuel_options,
    value=tuple(fuel_options),  # por defecto todos
    description="Combustibles",
    disabled=False
)

# Widget: rango de fechas
min_date, max_date = tidy["Fecha"].min(), tidy["Fecha"].max()
date_range = widgets.SelectionRangeSlider(
    options=[pd.Timestamp(d) for d in pd.date_range(min_date, max_date, freq="MS")],
    index=(0, len(pd.date_range(min_date, max_date, freq="MS"))-1),
    description='Rango',
    layout={'width': '95%'}
)

# Widget: selector de vistas a mostrar
view_options = [
    "Serie temporal (overlay)",
    "Composición stacked area",
    "Heatmap estacional",
    "Boxplots mensuales",
    "Treemap por año",
    "Barras por año (-> composición)",
    "Serie seleccionada -> Histograma (enlazado)",
    "Dispersión cambio mensual"
]
view_select = widgets.SelectMultiple(
    options=view_options,
    value=tuple(view_options),  # por defecto todas
    description="Vistas",
    disabled=False
)

# Contenedor de salida
out = widgets.Output()

def filter_data():
    fmin, fmax = date_range.value
    mask = (tidy["Fecha"] >= fmin) & (tidy["Fecha"] <= fmax) & (tidy["Combustible"].isin(list(fuel_multi.value)))
    return tidy.loc[mask].copy()

def make_time_series(df_sub):
    fig = px.line(df_sub, x="Fecha", y="Importaciones", color="Combustible",
                  title="Serie temporal por combustible (con range slider)")
    fig.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
    return fig

def make_stacked_area(df_sub):
    # Pivot para stacked area
    piv = df_sub.pivot_table(index="Fecha", columns="Combustible", values="Importaciones", aggfunc="sum").fillna(0)
    fig = go.Figure()
    for i, col in enumerate(piv.columns):
        fig.add_traces(go.Scatter(x=piv.index, y=piv[col], stackgroup='one', name=col, mode="lines"))
    fig.update_layout(title="Composición por combustible (área apilada)",
                      xaxis=dict(rangeslider=dict(visible=True)))
    return fig

def make_seasonal_heatmap(df_sub):
    tmp = df_sub.copy()
    tmp["Año"] = tmp["Fecha"].dt.year
    tmp["Mes"] = tmp["Fecha"].dt.month
    # Promedio por año-mes y combustible; si hay más de uno seleccionado, mostrar el total
    agg = tmp.groupby(["Año","Mes"], as_index=False)["Importaciones"].sum()
    fig = px.imshow(
        agg.pivot(index="Año", columns="Mes", values="Importaciones"),
        aspect="auto", origin="lower",
        title="Calor estacional (Año vs Mes) — suma de combustibles seleccionados"
    )
    return fig

def make_boxplots(df_sub):
    tmp = df_sub.copy()
    tmp["MesNombre"] = tmp["Fecha"].dt.month_name(locale="es_ES") if hasattr(pd.Series.dt, "month_name") else tmp["Fecha"].dt.month
    fig = px.box(tmp, x="Combustible", y="Importaciones", color="Combustible", points="all",
                 title="Distribución de importaciones por combustible (boxplot)")
    return fig

def make_treemap(df_sub):
    tmp = df_sub.copy()
    tmp["Año"] = tmp["Fecha"].dt.year
    agg = tmp.groupby(["Año","Combustible"], as_index=False)["Importaciones"].sum()
    fig = px.treemap(agg, path=["Año","Combustible"], values="Importaciones", title="Treemap por Año y Combustible")
    return fig

def make_year_bar_and_composition(df_sub):
    # Figura enlazada 1: barras por año (click actualiza composición por combustible)
    tmp = df_sub.copy()
    tmp["Año"] = tmp["Fecha"].dt.year
    year_agg = tmp.groupby("Año", as_index=False)["Importaciones"].sum()

    bar_year = go.FigureWidget(px.bar(year_agg, x="Año", y="Importaciones", title="Total por año (clic para ver composición)"))
    comp = go.FigureWidget()  # se rellenará con callback

    def update_composition(clicked_year=None):
        comp.data = ()
        if clicked_year is None and len(year_agg)>0:
            clicked_year = int(year_agg.iloc[-1]["Año"])  # último año por defecto
        sub = tmp[tmp["Año"]==clicked_year]
        sub_agg = sub.groupby("Combustible", as_index=False)["Importaciones"].sum()
        with comp.batch_update():
            for _, row in sub_agg.iterrows():
                comp.add_bar(x=[row["Combustible"]], y=[row["Importaciones"]], name=row["Combustible"])
            comp.update_layout(title=f"Composición por combustible — Año {clicked_year}",
                               xaxis_title="", yaxis_title="Importaciones", showlegend=False)

    # Callback: click en barras
    def handle_click(trace, points, state):
        if points.point_inds:
            idx = points.point_inds[0]
            clicked_year = int(year_agg.iloc[idx]["Año"])
            update_composition(clicked_year)

    bar_year.data[0].on_click(handle_click)
    update_composition()  # inicial

    return bar_year, comp

def make_linked_ts_hist(df_sub):
    # Figura enlazada 2: selección en serie temporal -> histograma
    piv = df_sub.pivot_table(index="Fecha", columns="Combustible", values="Importaciones", aggfunc="sum").fillna(0)
    # Sumamos combustibles seleccionados para vista agregada
    ts = piv.sum(axis=1)

    ts_fig = go.FigureWidget(px.line(x=ts.index, y=ts.values, labels={"x":"Fecha","y":"Importaciones"},
                                     title="Serie temporal agregada (selecciona un rango con zoom o lasso)"))

    hist_fig = go.FigureWidget(px.histogram(x=ts.values, nbins=30, labels={"x":"Importaciones"},
                                            title="Histograma del rango seleccionado"))

    def update_hist(xmin=None, xmax=None):
        vals = ts.values
        idx = np.ones_like(vals, dtype=bool)
        if xmin is not None and xmax is not None:
            idx = (ts.index >= xmin) & (ts.index <= xmax)
        with hist_fig.batch_update():
            hist_fig.data = ()
            hist_fig.add_histogram(x=ts.values[idx], nbinsx=30)
            hist_fig.update_layout(title="Histograma del rango seleccionado")

    # Callback: usar cambios en el eje x (zoom/relayout)
    def on_relayout(layout, trace, points, state):
        x_range = layout.get("xaxis.range")
        if x_range and isinstance(x_range, list) and len(x_range)==2:
            xmin = pd.to_datetime(x_range[0])
            xmax = pd.to_datetime(x_range[1])
            update_hist(xmin, xmax)

    ts_fig.layout.on_change(on_relayout, "xaxis")

    return ts_fig, hist_fig

def make_mom_scatter(df_sub):
    # Dispersión del cambio mensual (diferencia)
    tmp = df_sub.copy().pivot_table(index="Fecha", columns="Combustible", values="Importaciones", aggfunc="sum").fillna(0)
    series = tmp.sum(axis=1)
    chg = series.diff()
    fig = px.scatter(x=series.index, y=chg.values, labels={"x":"Fecha","y":"Δ Importaciones"},
                     title="Cambio mensual de las importaciones (Δ)")
    return fig


def refresh(*args):
    out.clear_output(wait=True)
    df_sub = filter_data()

    figs_to_show = []

    if "Serie temporal (overlay)" in view_select.value:
        figs_to_show.append(make_time_series(df_sub))
    if "Composición stacked area" in view_select.value:
        figs_to_show.append(make_stacked_area(df_sub))
    if "Heatmap estacional" in view_select.value:
        figs_to_show.append(make_seasonal_heatmap(df_sub))
    if "Boxplots mensuales" in view_select.value:
        figs_to_show.append(make_boxplots(df_sub))
    if "Treemap por año" in view_select.value:
        figs_to_show.append(make_treemap(df_sub))
    if "Barras por año (-> composición)" in view_select.value:
        bar_year, comp = make_year_bar_and_composition(df_sub)
        figs_to_show.append(bar_year); figs_to_show.append(comp)
    if "Serie seleccionada -> Histograma (enlazado)" in view_select.value:
        ts_fig, hist_fig = make_linked_ts_hist(df_sub)
        figs_to_show.append(ts_fig); figs_to_show.append(hist_fig)
    if "Dispersión cambio mensual" in view_select.value:
        figs_to_show.append(make_mom_scatter(df_sub))

    with out:
        clear_output(wait=True)
        for fig in figs_to_show:
            display(fig)

# Disparar por primera vez y conectar eventos
fuel_multi.observe(refresh, names="value")
date_range.observe(refresh, names="value")
view_select.observe(refresh, names="value")

ui = widgets.VBox([widgets.HBox([fuel_multi]), date_range, view_select])
display(ui)
refresh()
display(out)



## Modelado: 3+ modelos simples de predicción

Se evalúan al menos **tres modelos** para pronosticar importaciones por combustible (serie mensual):
1. **Naive estacional (lag 12)** — compara contra el valor del mismo mes del año anterior.  
2. **Media móvil (ventana 3)** — suaviza la serie y usa el último promedio como pronóstico.  
3. **Holt–Winters aditivo** — componentes de nivel, tendencia y estacionalidad (12).  
4. *(Opcional)* **RandomForest Regressor** con variables de calendario.

Se utiliza como **set de prueba** los **últimos 12 meses**. Se reportan **MAE**, **RMSE**, **MAPE%** y **R²**. Además, se incluye una **learning curve** para el modelo de RandomForest.


In [ ]:

# Widget: selección de combustible para modelado
fuel_model = widgets.Dropdown(options=sorted(tidy["Combustible"].unique()), value=sorted(tidy["Combustible"].unique())[0],
                              description="Combustible:")

# Widget: selección de modelos a comparar
model_list = ["Naive-12", "MA(3)", "Holt-Winters", "RandomForest"]
model_check = widgets.SelectMultiple(options=model_list, value=("Naive-12","MA(3)","Holt-Winters"), description="Modelos")

# Salida
out_models = widgets.Output()

def run_models(*args):
    out_models.clear_output(wait=True)
    # Serie del combustible seleccionado
    s = tidy.loc[tidy["Combustible"]==fuel_model.value, ["Fecha","Importaciones"]].set_index("Fecha").sort_index()["Importaciones"]
    # Asegurar frecuencia mensual
    s = s.asfreq("MS")
    train, test = split_series(s, test_months=12)

    # Ejecutar modelos seleccionados
    preds_dict = {}
    start_date = s.index.min()

    if "Naive-12" in model_check.value:
        preds_dict["Naive-12"] = forecast_naive_seasonal(train, test, seasonal_lag=12)
    if "MA(3)" in model_check.value:
        preds_dict["MA(3)"] = forecast_moving_average(train, test, window=3)
    if "Holt-Winters" in model_check.value:
        preds_dict["Holt-Winters"] = forecast_holt_winters(train, test)
    if "RandomForest" in model_check.value:
        preds_dict["RandomForest"] = forecast_random_forest(train, test, start_date=start_date)

    # Tabla de métricas
    rows = []
    for m, yhat in preds_dict.items():
        mtr = metrics(test.values, yhat)
        mtr["Modelo"] = m
        rows.append(mtr)
    met_df = pd.DataFrame(rows).set_index("Modelo").sort_values("RMSE")

    # Figura: actual vs predicciones
    fig = go.Figure()
    fig.add_scatter(x=train.index, y=train.values, mode="lines", name="Train")
    fig.add_scatter(x=test.index, y=test.values, mode="lines+markers", name="Test (real)")
    for m, yhat in preds_dict.items():
        fig.add_scatter(x=test.index, y=yhat, mode="lines+markers", name=f"Pred: {m}")
    fig.update_layout(title=f"Pronósticos — {fuel_model.value} (últimos 12 meses como prueba)",
                      xaxis_title="Fecha", yaxis_title="Importaciones")

    # Learning curve para RF si se seleccionó
    lc_fig = None
    if "RandomForest" in model_check.value:
        lc_fig = make_learning_curve(s.dropna(), start_date=start_date)

    with out_models:
        display(fig)
        display(px.imshow(met_df.round(3), text_auto=True, aspect="auto",
                          title="Comparación de métricas (menor es mejor para MAE/RMSE/MAPE)"))
        if lc_fig is not None:
            display(lc_fig)

fuel_model.observe(run_models, names="value")
model_check.observe(run_models, names="value")
display(widgets.HBox([fuel_model, model_check]))
run_models()
display(out_models)



## Exportar (opcional)

Puede exportar visualizaciones y resultados a **HTML auto-contenido**, útil para compartir con directivos.  
> Nota: la exportación a un *único* HTML con todas las vistas no siempre conserva callbacks entre `FigureWidget`. Una alternativa es exportar vistas clave de forma individual.


In [ ]:

# Ejemplo: exportar una figura clave a HTML
# from plotly.io import write_html
# df_sub = tidy.copy()
# fig_key = px.line(df_sub, x="Fecha", y="Importaciones", color="Combustible",
#                   title="Serie temporal por combustible")
# write_html(fig_key, file="serie_temporal.html", auto_open=False, include_plotlyjs="cdn")
# print("Archivo 'serie_temporal.html' generado en el directorio actual.")
